In [ ]:
import pandas as pd
import collections

from etl_gestores import etl as etlgest
from etl_gestores import etl_nota
from etl_idesp import etl as etlidesp

In [ ]:
#https://dados.educacao.sp.gov.br/search/field_topic/desafios-seduc-164

In [ ]:
df_gestao = etl_nota()

In [ ]:
df_gestao.head(3)

In [ ]:
df_gestores = etlgest()

In [ ]:
df_idesp = etlidesp()

In [ ]:
result = pd.merge(df_idesp,
                  df_gestores,
                  on='id_gestao')

In [ ]:
nota_gestor = {}
nota_gestor['id_gestor'] = []
nota_gestor['nota_var'] = []
nota_gestor['n_atuacoes'] = []
for nome, dados in result.groupby('id_gestor'):
    nota_gestor['id_gestor'].append(nome)
    nota_gestor['nota_var'].append(dados.variacao.mean())
    nota_gestor['n_atuacoes'].append(len(dados))

nota_gestor = pd.DataFrame(data=nota_gestor)

In [ ]:
nota_gestor.head(3)

In [ ]:
result2 = pd.merge(nota_gestor,
                   df_gestores,
                   on='id_gestor')

In [ ]:
nota_gestor[nota_gestor.id_gestor == 'NAO']

In [ ]:
result2[result2.id_gestor == 403084]

# FATO
- Avaliacao do Saresp
- Ano
- Valor (medprof)
- Dim disciplina (serie/periodo/conteudo) - TODO: incluir o professor aqui.
- Dim equipe de gestores - tabela com Ids e uma auxiliar de detalhamentos
- Dim escola

# FUNCOES

# CARREGAMENTO DOS DADOS

In [ ]:
# GESTORES
arquivo = 'dados\lideres\lideres_DIRETORES DE ESCOLA.csv'
diretor = pd.read_csv(arquivo, sep=';')
arquivo = 'dados\lideres\lideres_VICE_DIRETOR.csv'
vice = pd.read_csv(arquivo, sep=';')
arquivo = 'dados\lideres\lideres_PROFESSOR_COORDENADOR.csv'
coordenador = pd.read_csv(arquivo, sep=';')
vice = vice.drop('Unnamed: 16',axis=1)

In [ ]:
df_gestores = etl()

In [ ]:
df_gestores

### ESTUDO DAS VARIACOES

In [ ]:
idesp_group[idesp_group.variacao > 200]

In [ ]:
df_idesp[df_idesp.loc[:,'CODIGO CIE'] == 40551]

# GERANDO DIMENSAO GESTORES
- cd_escola
- ano
- diretoria
- id_diretor - detalhamento aparece em outro lugar
- id_vice
- id_coordenador

In [ ]:
LISTA_ANOS = ['2014', '2015', '2016', '2017', '2018']
def foi_gestor(ano):
    return ano == 'SIM'

In [ ]:
def limpeza(tabela_extraida, funcao_gestor, df_gestao):
    for _, row in tabela_extraida.iterrows():
        for ANO in LISTA_ANOS:
            if foi_gestor(row.loc[ANO]):
                df_gestao['id_gestao'].append('esc-' + str(row.CD_ESCOLA) + '-ano-' + str(ANO))
                df_gestao['id_escola'].append(row.CD_ESCOLA)
                df_gestao['id_gestor'].append(row.id_interno)
                df_gestao['ano_gestao'].append(ANO)
                df_gestao['diretoria'].append(row.DIRETORIA)
                df_gestao['funcao'].append(funcao_gestor)
    return df_gestao

In [ ]:
df_gestao = {}
df_gestao['id_gestao'] = []
df_gestao['id_escola'] = []
df_gestao['id_gestor'] = []
df_gestao['ano_gestao'] = []
df_gestao['diretoria'] = []
df_gestao['funcao'] = []

df_gestao = limpeza(diretor, 'DIRETOR', df_gestao)
df_gestao = limpeza(vice, 'VICE', df_gestao)
df_gestao = limpeza(coordenador, 'COORDENADOR', df_gestao)

In [ ]:
df_gestao = pd.DataFrame(df_gestao)

In [ ]:
df_gestao = {}
df_gestao['id_gestao'] = []
df_gestao['id_escola'] = []
df_gestao['id_gestor'] = []
df_gestao['ano_gestao'] = []
df_gestao['diretoria'] = []
df_gestao['funcao'] = []

for _, row in diretor.iterrows():
    for ANO in LISTA_ANOS:
        if foi_gestor(row.loc[ANO]):
            df_gestao['id_gestao'].append('esc-' + str(row.CD_ESCOLA) + '-ano-' + str(ANO))
            df_gestao['id_escola'].append(row.CD_ESCOLA)
            df_gestao['id_gestor'].append(row.id_interno)
            df_gestao['ano_gestao'].append(ANO)
            df_gestao['diretoria'].append(row.DIRETORIA)
            df_gestao['funcao'].append('DIRETOR')

for _, row in vice.iterrows():
    for ANO in LISTA_ANOS:
        if foi_gestor(row.loc[ANO]):
            df_gestao['id_gestao'].append('esc-' + str(row.CD_ESCOLA) + '-ano-' + str(ANO))
            df_gestao['id_escola'].append(row.CD_ESCOLA)
            df_gestao['id_gestor'].append(row.id_interno)
            df_gestao['ano_gestao'].append(ANO)
            df_gestao['diretoria'].append(row.DIRETORIA)
            df_gestao['funcao'].append('VICE')
            
for _, row in coordenador.iterrows():
    for ANO in LISTA_ANOS:
        if foi_gestor(row.loc[ANO]):
            df_gestao['id_gestao'].append('esc-' + str(row.CD_ESCOLA) + '-ano-' + str(ANO))
            df_gestao['id_escola'].append(row.CD_ESCOLA)
            df_gestao['id_gestor'].append(row.id_interno)
            df_gestao['ano_gestao'].append(ANO)
            df_gestao['diretoria'].append(row.DIRETORIA)
            df_gestao['funcao'].append('COORDENADOR')
            
df_gestao = pd.DataFrame(df_gestao)

### VERIFICANDO QUANTIDADE DE TROCAS DE GESTORES

In [ ]:
k = 0
contagem_gestores = collections.Counter(df_gestao.id_gestao)
for id_gestao in contagem_gestores:
    if contagem_gestores[id_gestao] == 3:
        k += 1

In [ ]:
df_gestao[df_gestao.id_gestao == 'esc-472505-ano-2014']

# GESTAO + IDESP

In [ ]:
cont_gestores = {}
cont_gestores['id_gestao'] = []
cont_gestores['id_escola'] = []
cont_gestores['n_gestores'] = []

for nome, dados in df_gestao.groupby('id_gestao'):
    n_gestores = len(dados)
    cont_gestores['id_gestao'].append(nome)
    cont_gestores['id_escola'].append(dados.id_escola.iloc[0])
    cont_gestores['n_gestores'].append(n_gestores)
    
cont_gestores = pd.DataFrame(data=cont_gestores)
cont_gestores_aux = {}
cont_gestores_aux['id_escola'] = []
cont_gestores_aux['mediana'] = []
for nome, dados in cont_gestores.groupby('id_escola'):
    cont_gestores_aux['id_escola'].append(dados.id_escola.iloc[0])
    cont_gestores_aux['mediana'].append(dados.n_gestores.median())

cont_gestores_aux = pd.DataFrame(data=cont_gestores_aux)
cont_gestores = pd.merge(cont_gestores,
                  cont_gestores_aux,
                  on='id_escola')
cont_gestores['var_gestores'] = result.n_gestores - result.mediana

result = pd.merge(idesp_group,
                  cont_gestores,
                  on='id_gestao')
result.loc[:,['variacao','var_gestores']].corr()

# DIFERENTE DO QUE EU PENSAVA, PARECE QUE ESSE NUMERO ALTO DE GESTORES
# MAIS OU MENOS NORMAL E NAO IMPACTA MUITO OS RESULTADOS

# NOTAS DOS GESTORES

In [ ]:
result = pd.merge(idesp_group,
                  df_gestao,
                  on='id_gestao')

In [ ]:
result.head(5)

In [ ]:
nota_gestor = {}
nota_gestor['id_gestor'] = []
nota_gestor['nota_var'] = []
nota_gestor['n_atuacoes'] = []
for nome, dados in result.groupby('id_gestor'):
    nota_gestor['id_gestor'].append(nome)
    nota_gestor['nota_var'].append(dados.variacao.mean())
    nota_gestor['n_atuacoes'].append(len(dados))

nota_gestor = pd.DataFrame(data=nota_gestor)

In [ ]:
nota_gestor.nota_var.describe()

In [ ]:
nota_gestor.n_atuacoes.describe()

In [ ]:
nota_gestor = nota_gestor[nota_gestor.n_atuacoes >= 2]

In [ ]:
nota_gestor.head(5)

In [ ]:
result[result.id_gestor == 355299]

In [ ]:
dados

# DIMENSAO SARESP
- Substituida pela dimensao IDESP

In [ ]:
def limpa_serie_ano(serie_ano):
    if serie_ano == '9Âº Ano EF' or serie_ano == '9º Ano EF':
        return 'EF-9'
    elif serie_ano == '7Âº Ano EF':
        return 'EF-7'
    elif serie_ano == 'EM-3Âª sÃ©rie' or serie_ano == 'EM-3ª série':
        return 'EM-3'
    elif serie_ano == '3Âº Ano EF' or serie_ano == '3º Ano EF':
        return 'EF-3'
    elif serie_ano == '5Âº Ano EF' or serie_ano == '5º Ano EF':
        return 'EF-5'
    
def limpa_ds_comp(ds_comp):
    if ds_comp == 'LÃ\x8dNGUA PORTUGUESA' or ds_comp == 'LÍNGUA PORTUGUESA':
        return 'PORTUGUES'
    elif ds_comp == 'MATEMÃ\x81TICA' or ds_comp == 'MATEMÁTICA':
        return 'MATEMATICA'
    elif ds_comp == 'CIÃ\x8aNCIAS':
        return 'CIENCIAS'    
    
def enulo(valor):
    return str(valor) == 'nan'   

In [ ]:
# SARESP
arquivo = 'dados/saresp/SARESP_escolas_2014.csv'
df_2014 = pd.read_csv(arquivo, sep=';', encoding='latin-1')
df_2014['ano'] = 2014
arquivo = 'dados/saresp/SARESP_escolas_2015.csv'
df_2015 = pd.read_csv(arquivo, sep=';', encoding='latin-1')
df_2015['ano'] = 2015
arquivo = 'dados/saresp/SARESP_escolas_2016.csv'
df_2016 = pd.read_csv(arquivo, sep=';', encoding='latin-1')
df_2016['ano'] = 2016
arquivo = 'dados/saresp/SARESP_escolas_2017.csv'
df_2017 = pd.read_csv(arquivo, sep=';', encoding='latin-1')
df_2017['ano'] = 2017
arquivo = 'dados/saresp/SARESP_escolas_2018.csv'
df_2018 = pd.read_csv(arquivo, sep=';', encoding='latin-1')
df_2018['ano'] = 2018
saresp = df_2014.append(df_2015).append(df_2016).append(df_2017).append(df_2018)
saresp = saresp[saresp.medprof.notnull()] # nao estou interessado se nao tiver o valor do saresp

In [ ]:
saresp.SERIE_ANO = saresp.SERIE_ANO.apply(limpa_serie_ano)
saresp.ds_comp = saresp.ds_comp.apply(limpa_ds_comp)
saresp.medprof = saresp.medprof \
                .apply(lambda x: str(x).replace('.','').replace(',','.')) \
                .astype(float) 

df_saresp = {}
df_saresp['id_gestao'] = []
df_saresp['id_escola'] = []
df_saresp['ano_gestao'] = []
df_saresp['serie_ano'] = []
df_saresp['periodo'] = []
df_saresp['disciplina'] = []
df_saresp['medprofici'] = []

for _, row in saresp.iterrows():
    df_saresp['id_gestao'].append('esc-' + str(row.CODESC) + '-ano-' + str(row.ano))
    df_saresp['id_escola'].append(row.CODESC)
    df_saresp['ano_gestao'].append(row.ano)
    df_saresp['serie_ano'].append(row.SERIE_ANO)
    df_saresp['periodo'].append(row.periodo)
    df_saresp['disciplina'].append(row.ds_comp)
    df_saresp['medprofici'].append(row.medprof)

df_saresp = pd.DataFrame(data=df_saresp) 

saresp_medias = {}
saresp_medias['id_gestao'] = []
saresp_medias['media'] = []
saresp_medias['id_escola'] = []
for nome, dados in df_saresp.groupby('id_gestao'):
    saresp_medias['id_gestao'].append(nome)
    saresp_medias['media'].append(dados.medprofici.mean())
    saresp_medias['id_escola'].append(dados.id_escola.iloc[0])

saresp_medias = pd.DataFrame(data = saresp_medias)

In [1]:
import pandas as pd
import collections

import etl_gestores
import etl_idesp

In [2]:
dados = etl_gestores.etl()

In [3]:
dados.head(3)

,id_gestao,id_escola,id_gestor,ano_gestao,diretoria,funcao
0,esc-902724-ano-2017,902724,288268,2017,LESTE 3,DIRETOR
1,esc-902724-ano-2018,902724,288268,2018,LESTE 3,DIRETOR
2,esc-13006-ano-2015,13006,385732,2015,GUARATINGUETA,DIRETOR


In [4]:
df_idesp = etl_idesp.etl()

In [5]:
mergeidesp = pd.merge(df_idesp, dados, on='id_gestao')

In [6]:
mergeidesp.head(3)

,id_gestao,id_escola_x,ano_gestao_x,nota_ano,variacao,anterior_nulo,atual_nulo,unidades_diretoria,nome_diretoria,id_escola_y,id_gestor,ano_gestao_y,diretoria,funcao
0,esc-10005-ano-2014,10005,2014,3.48,10.828025,0,1,133,ITAPECERICA DA SERRA,10005,326625,2014,ITAPECERICA DA SERRA,DIRETOR
1,esc-10005-ano-2014,10005,2014,3.48,10.828025,0,1,133,ITAPECERICA DA SERRA,10005,157667,2014,ITAPECERICA DA SERRA,VICE
2,esc-10005-ano-2014,10005,2014,3.48,10.828025,0,1,133,ITAPECERICA DA SERRA,10005,314895,2014,ITAPECERICA DA SERRA,VICE


In [7]:
mergeidesp = mergeidesp.dropna(subset=['nota_ano'])
mergeidesp = mergeidesp.rename({'ano_gestao_x': 'ano_gestao'}, axis=1)
mergeidesp.ano_gestao = mergeidesp.ano_gestao.astype(int)

In [8]:
mergeidesp[mergeidesp.id_gestor == 326625].head(3)
# O Gestor X atuou em a,b,c anos nas seguintes escolas.
# A nota nestes anos foi Z

,id_gestao,id_escola_x,ano_gestao,nota_ano,variacao,anterior_nulo,atual_nulo,unidades_diretoria,nome_diretoria,id_escola_y,id_gestor,ano_gestao_y,diretoria,funcao
0,esc-10005-ano-2014,10005,2014,3.48,10.828025,0,1,133,ITAPECERICA DA SERRA,10005,326625,2014,ITAPECERICA DA SERRA,DIRETOR
5,esc-10005-ano-2015,10005,2015,3.78,8.620690,0,1,133,ITAPECERICA DA SERRA,10005,326625,2015,ITAPECERICA DA SERRA,DIRETOR
11,esc-10005-ano-2016,10005,2016,3.26,-13.756614,0,1,133,ITAPECERICA DA SERRA,10005,326625,2016,ITAPECERICA DA SERRA,DIRETOR


In [9]:
LISTA_ANOS = [2014, 2015, 2016, 2017, 2018]
adic_media = {}
adic_media['id_gestao'] = []
adic_media['media_variacao'] = []
adic_media['media_gestor'] = []

for nome,dados in mergeidesp.groupby('id_gestor'):
    nota_total = 0
    variacao_total = 0
    contagem = 0
    lista_anos = dados.ano_gestao.tolist()
    lista_anos.sort()
    primeira_gestao = lista_anos[0]
    nota_ano_antes = 0
    variacao_ano_antes = 0
    for ano in lista_anos:
        if ano == primeira_gestao:
            row = dados[dados.ano_gestao == primeira_gestao].iloc[0]
            nota_ano_antes = row.nota_ano
            variacao_ano_antes = row.variacao
            continue
            #nota_anterior = row.nota_ano * (1 - row.variacao/100)
            #variacao_anterior = 0
        else:
            nota_anterior = nota_ano_antes
            variacao_anterior = variacao_ano_antes
            row = dados[dados.ano_gestao == ano].iloc[0]
            nota_ano_antes = row.nota_ano
            variacao_ano_antes = row.variacao
            
        nota_total += nota_anterior
        variacao_total += variacao_anterior
        contagem += 1

        row = dados[dados.ano_gestao == ano].iloc[0]
        adic_media['id_gestao'].append(row.id_gestao)
        adic_media['media_gestor'].append(nota_total / contagem)
        adic_media['media_variacao'].append(variacao_total / contagem)
        
adic_media = pd.DataFrame(data=adic_media)

In [10]:
mergeidesp2 = pd.merge(mergeidesp, adic_media, on='id_gestao', how='left')

In [11]:
mergeidesp2.loc[:,['nota_ano','media_variacao','media_gestor']].corr()

,nota_ano,media_variacao,media_gestor
nota_ano,1.000000,0.024101,0.822983
media_variacao,0.024101,1.000000,0.083194
media_gestor,0.822983,0.083194,1.000000


In [12]:
mergeidesp2.media_variacao.isnull().sum()

34327